Tıtanic için,
Modelinizin performansı çapraz doğrulama yaparak ve farklı metrikler kullanarak değerlendirin.
Hiperparemetre ayarlaması yaparak en uygun parametrelere sahip modeli belirleyin.

Modelinizin performansına bu derste öğrendiğiniz metriklerlerle değerlendiriniz. ROC eğrisi ve Hassasiyet/Duyarlılık Eğrisini farklı "c" değerleri için çizdirin.

In [204]:
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


import warnings
warnings.filterwarnings('ignore')


In [267]:
df = pd.read_csv("https://storage.googleapis.com/kagglesdsdata/competitions/3136/26502/train.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1619022461&Signature=YYgjqe7orqvSPdr9iSz2R3XADmpPrI1Ee4qZzcTQNyovPepMdPe5EZ1z%2FLHWLdKS7zmarnBQ8zwlPInvoFcrq%2BjltMdOAw5%2BE%2FpMZSNdSsaPIFQwoL8%2BJmdHWnNnRvV%2BPFUwROEGaSOmKGomrGp4gLZk2z6RyowEf9n88XSph65VayLBNz2QQaByWc5HnIB5xDLPuxX9mf9Y95lQwVhyUavkbyvh7mcihOh9DLddrGR47O4VaEQyc%2BxzyFO2i9d9IjkH7D6eoikMyZ3WZtLfz2%2BEZ1thjRaY1oeOTDC30S78wfoGLpW5yopB8XzJCc76QeWRaALVfYHKDuAOnJ7DKA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv")

In [121]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [268]:
df["Sex"]=pd.get_dummies(df["Sex"],drop_first=True)

In [269]:
df=df.drop("Name",axis=1)

In [125]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [270]:
df=df.drop("Cabin",axis=1)

In [271]:
df["Age"]=df["Age"].fillna(df["Age"].mean())


In [128]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

In [272]:
df["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [130]:
print("How many 'S' on embarked column : " , df[df['Embarked'] == 'S'].shape[0])
print("How many 'C' on embarked column : " ,df[df['Embarked'] == 'C'].shape[0])
print("How many 'Q' on embarked column : ",df[df['Embarked'] == 'Q'].shape[0])

How many 'S' on embarked column :  644
How many 'C' on embarked column :  168
How many 'Q' on embarked column :  77


In [273]:
df = df.fillna({'Embarked' : 'S'})

In [274]:
df=df.drop("Ticket",axis=1)

In [275]:
y=df["Survived"]
x=df.drop("Survived",axis=1)

In [276]:
x=df.drop("PassengerId",axis=1)
x=df.drop("Fare",axis=1)

In [244]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    uint8  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Embarked     891 non-null    object 
dtypes: float64(1), int64(5), object(1), uint8(1)
memory usage: 49.7+ KB


In [277]:
df2=pd.get_dummies(x["Embarked"])
x=pd.concat([x,df2],axis=1)
x


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,C,Q,S
0,1,0,3,1,22.000000,1,0,S,0,0,1
1,2,1,1,0,38.000000,1,0,C,1,0,0
2,3,1,3,0,26.000000,0,0,S,0,0,1
3,4,1,1,0,35.000000,1,0,S,0,0,1
4,5,0,3,1,35.000000,0,0,S,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.000000,0,0,S,0,0,1
887,888,1,1,0,19.000000,0,0,S,0,0,1
888,889,0,3,0,29.699118,1,2,S,0,0,1
889,890,1,1,1,26.000000,0,0,C,1,0,0


In [278]:
x=x.drop("Embarked",axis=1)
x=x.drop("Survived",axis=1)

In [279]:
x

,PassengerId,Pclass,Sex,Age,SibSp,Parch,C,Q,S
0,1,3,1,22.000000,1,0,0,0,1
1,2,1,0,38.000000,1,0,1,0,0
2,3,3,0,26.000000,0,0,0,0,1
3,4,1,0,35.000000,1,0,0,0,1
4,5,3,1,35.000000,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
886,887,2,1,27.000000,0,0,0,0,1
887,888,1,0,19.000000,0,0,0,0,1
888,889,3,0,29.699118,1,2,0,0,1
889,890,1,1,26.000000,0,0,1,0,0


In [280]:
X_egitim, X_test, y_egitim, y_test = train_test_split(x, y, test_size=0.2, stratify=y)

print("Tüm veri kümesi '0' yüzdesi : %{:.0f} ".format(len(y[y==0])/len(y)*100))
print("Test verisi '0' yüzdesi     : %{:.0f} ".format(len(y_test[y_test==0])/len(y_test)*100))
print("Eğitim verisi '0' yüzdesi   : %{:.0f} ".format(len(y_egitim[y_egitim==0])/len(y_egitim)*100))

Tüm veri kümesi '0' yüzdesi : %62 
Test verisi '0' yüzdesi     : %61 
Eğitim verisi '0' yüzdesi   : %62 


In [286]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_egitim, y_egitim)
tahmin_eğitim = model.predict(X_egitim)
tahmin_test = model.predict(X_test)
print(model.score(X_test, y_test))
print(model.score(X_egitim,y_egitim))

0.7486033519553073
0.8047752808988764


In [294]:
from sklearn.model_selection import cross_validate, cross_val_score

In [324]:
lrm = LogisticRegression()
cv = cross_validate(estimator=lrm,
                     X=x,
                     y=y,
                     cv=5,
                    )
print('Test Skorları            : ', cv['test_score'], sep = '\n')
print("-"*50)
print('Eğitim Skorları          : ', cv['train_score'], sep = '\n')

#burada eğitiim verilerinin doğruluk oranlarını gösteremedim hatayı anlayamadım.

Test Skorları            : 
[0.75418994 0.80337079 0.79213483 0.76404494 0.79775281]
--------------------------------------------------


KeyError: 'train_score'

In [325]:
print('Test Kümesi   Ortalaması : ', cv['test_score'].mean())

Test Kümesi   Ortalaması :  0.7822986629841191


In [295]:
print(cross_val_score(model,x,y,cv=5))

[0.75418994 0.80337079 0.79213483 0.76404494 0.79775281]


In [321]:
accuracies=cross_val_score(estimator=model,X=x,y=y,cv=10)
print(" Test kümesi ortalaması",accuracies.mean()*100)

 Test kümesi ortalaması 78.45443196004992


In [326]:
cv = cross_validate(estimator=lrm, 
                     X=x,
                     y=y,
                     cv=10,
                     scoring = ['accuracy', 'precision', 'r2']
                    )

In [327]:
print('Test Kümesi Doğruluk Ortalaması     : {:.2f}'.format(cv['test_accuracy'].mean()))
print('Test Kümesi R-kare  Ortalaması      : {:.2f}'.format(cv['test_r2'].mean()))
print('Test Kümesi Hassasiyet Ortalaması   : {:.2f}'.format(cv['test_precision'].mean()))
print('Eğitim Kümesi Doğruluk Ortalaması   : {:.2f}'.format(cv['train_accuracy'].mean()))
print('Eğitim Kümesi R-kare  Ortalaması    : {:.2f}'.format(cv['train_r2'].mean()))
print('Eğitim Kümesi Hassasiyet Ortalaması : {:.2f}'.format(cv['train_precision'].mean()))

Test Kümesi Doğruluk Ortalaması     : 0.78
Test Kümesi R-kare  Ortalaması      : 0.09
Test Kümesi Hassasiyet Ortalaması   : 0.76


KeyError: 'train_accuracy'

Hiperparemetre ayarlaması yaparak en uygun parametrelere sahip modeli belirleyin.

In [328]:
logreg = LogisticRegression()
print(logreg.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [329]:
parametreler = {"C": [10 ** x for x in range (-5, 5, 1)],
                "penalty": ['l1', 'l2']
                }

In [331]:
from sklearn.model_selection import GridSearchCV
grid_cv = GridSearchCV(estimator=logreg,
                       param_grid = parametreler,
                       cv = 10
                      )
grid_cv.fit(x, y)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000, 10000],
                         'penalty': ['l1', 'l2']})

In [332]:
print("En iyi parametreler : ", grid_cv.best_params_)
print("En iyi skor         : ", grid_cv.best_score_)

En iyi parametreler :  {'C': 100, 'penalty': 'l2'}
En iyi skor         :  0.7935205992509363


In [336]:
sonuçlar = grid_cv.cv_results_
df3 = pd.DataFrame(sonuçlar)
df3.head()


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001203,4.010925e-04,0.000000,0.000000,1e-05,l1,"{'C': 1e-05, 'penalty': 'l1'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20
1,0.017048,2.174341e-03,0.002607,0.000666,1e-05,l2,"{'C': 1e-05, 'penalty': 'l2'}",0.611111,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.606742,0.616167,0.00375,8
2,0.001103,3.014276e-04,0.000000,0.000000,0.0001,l1,"{'C': 0.0001, 'penalty': 'l1'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
3,0.020055,3.115185e-03,0.002156,0.000451,0.0001,l2,"{'C': 0.0001, 'penalty': 'l2'}",0.611111,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.606742,0.616167,0.00375,8
4,0.001003,5.883676e-07,0.000000,0.000000,0.001,l1,"{'C': 0.001, 'penalty': 'l1'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15


In [338]:
df3 = df3[['param_penalty','param_C', 'mean_test_score']]
df3 = df3.sort_values(by='mean_test_score', ascending = False)
df3

,param_penalty,param_C,mean_test_score
15,l2,100,0.793521
11,l2,1,0.784544
13,l2,10,0.782297
17,l2,1000,0.782297
9,l2,0.1,0.781161
19,l2,10000,0.780037
7,l2,0.01,0.753146
1,l2,1e-05,0.616167
3,l2,0.0001,0.616167
5,l2,0.001,0.616155


In [339]:
parametreler = {"C": [10 ** x for x in range (-5, 5, 1)],
                "penalty": ['l1', 'l2']
                }
from sklearn.model_selection import RandomizedSearchCV
rs_cv = RandomizedSearchCV(estimator=logreg,
                           param_distributions = parametreler,
                           cv = 10,
                           n_iter = 10,
                           random_state = 111,
                           scoring = 'precision'
                      )
rs_cv.fit(x, y)

RandomizedSearchCV(cv=10, estimator=LogisticRegression(),
                   param_distributions={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1,
                                              1, 10, 100, 1000, 10000],
                                        'penalty': ['l1', 'l2']},
                   random_state=111, scoring='precision')

In [340]:
print("En iyi parametreler        : ", rs_cv.best_params_)
print("Tüm hassasiyet değerleri   : ", rs_cv.cv_results_['mean_test_score'])
print("En iyi hassasiyet değeri   : ", rs_cv.best_score_)

En iyi parametreler        :  {'penalty': 'l2', 'C': 10}
Tüm hassasiyet değerleri   :  [0.         0.1        0.75873812 0.75699227        nan 0.74487161
        nan        nan        nan        nan]
En iyi hassasiyet değeri   :  0.7587381239450205
